<a href="https://colab.research.google.com/github/NitinGautam05/Plant-leaf-disease-identification/blob/main/Conv4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import zipfile
import io

# URL of the ZIP file
url = "https://data.mendeley.com/public-files/datasets/tywbtsjrjv/files/d5652a28-c1d8-4b76-97f3-72fb80f94efc/file_downloaded"

# Send an HTTP GET request to the URL
response = requests.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Open the ZIP file
    with zipfile.ZipFile(io.BytesIO(response.content), "r") as zip_ref:
        # Extract all the contents to a directory
        zip_ref.extractall("")
    print("Extraction successful.")
else:
    print("Failed to download the ZIP file.")

Extraction successful.


In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

data_dir = 'Plant_leave_diseases_dataset_without_augmentation'

# Define image dimensions and batch size
img_width, img_height = 256, 256
batch_size = 32

# Use ImageDataGenerator to load and preprocess images
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

test_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

print("Number of training samples:", train_generator.samples)
print("Number of validation samples:", validation_generator.samples)
print("Number of test samples:", test_generator.samples)

# Create Conv4 model
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(256, (3, 3), activation='relu'))  # Fourth convolutional layer
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(39, activation='softmax'))


# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()




Found 44371 images belonging to 39 classes.
Found 11077 images belonging to 39 classes.
Found 55448 images belonging to 39 classes.
Number of training samples: 44371
Number of validation samples: 11077
Number of test samples: 55448
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 127, 127, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 62, 62, 64)        0         
 g2D)                                                            
                      

In [ ]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)



Epoch 1/10
1386/1386 [==============================] - 140s 95ms/step - loss: 0.9564 - accuracy: 0.7239 - val_loss: 0.4984 - val_accuracy: 0.8457
Epoch 2/10
1386/1386 [==============================] - 149s 108ms/step - loss: 0.2869 - accuracy: 0.9074 - val_loss: 0.2766 - val_accuracy: 0.9119
Epoch 3/10
1386/1386 [==============================] - 131s 94ms/step - loss: 0.1649 - accuracy: 0.9459 - val_loss: 0.3048 - val_accuracy: 0.9106
Epoch 4/10
1386/1386 [==============================] - 130s 94ms/step - loss: 0.1253 - accuracy: 0.9592 - val_loss: 0.2459 - val_accuracy: 0.9289
Epoch 5/10
1386/1386 [==============================] - 130s 94ms/step - loss: 0.0948 - accuracy: 0.9687 - val_loss: 0.3214 - val_accuracy: 0.9197
Epoch 6/10
1386/1386 [==============================] - 138s 100ms/step - loss: 0.0792 - accuracy: 0.9748 - val_loss: 0.2496 - val_accuracy: 0.9382
Epoch 7/10
1386/1386 [==============================] - 128s 92ms/step - loss: 0.0655 - accuracy: 0.9792 - val_loss:

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.samples // batch_size)
print("Test accuracy:", test_acc)

# Predict the labels for the test set
y_pred = model.predict(test_generator, steps=test_generator.samples // batch_size)
y_true = test_generator.classes

# Convert predictions to class labels
y_pred_labels = np.argmax(y_pred, axis=1)

# Check lengths of y_true and y_pred_labels
# print(len(y_true), len(y_pred_labels))

# Predict the labels for the entire test set
y_pred_labels = []
for _ in range(test_generator.samples // batch_size):
    images, labels = test_generator.next()
    pred = model.predict(images)
    pred_labels = np.argmax(pred, axis=1)
    y_pred_labels.extend(pred_labels)

# Ensure that y_pred_labels has the same length as y_true
if len(y_true) == len(y_pred_labels):
    # Calculate precision, recall, and F1 score
    precision = precision_score(y_true, y_pred_labels, average='weighted')
    recall = recall_score(y_true, y_pred_labels, average='weighted')
    f1 = f1_score(y_true, y_pred_labels, average='weighted')
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 Score:", f1)
else:
    print("Lengths of y_true and y_pred_labels still do not match.")



1732/1732 [==============================] - 109s 63ms/step - loss: 0.0929 - accuracy: 0.9776
Test accuracy: 0.9775909185409546
1/1 [==============================] - 0s 29ms/step


KeyboardInterrupt: 